In [1]:
import ee
ee.Initialize() # uses ma.m@gmail.com
from radiantearth.api import API
import matplotlib.pyplot as plt
%matplotlib notebook
from IPython.display import Image
import folium

In [2]:
#Display thumbnail of classification output
#aoi = ""
#image = output.clip(aoi)
#theGeom = area_of_interest.geometry().getInfo()['coordinates']
#thumbnail = image.getThumbUrl({'min':0,'max':211,'size':'800','region':theGeom})
#Image(url=thumbnail)

THIS PART MAKES A CONNECTION TO THE 228 DATABSE TO CHECK FOR FEATURE
TO BE USED IN THE APPLICATION. ALSO, THIS IS WHERE WE CONFIGURE THE CONNECTION/LINK BETWEEN GEE PYTHON API AND WFP HQ DATABASE FOR STORAGE AND SHARING

In [3]:
#ConnectDB
import psycopg2
import psycopg2.extras
import getpass
import geopandas as gpd
from geopandas import GeoDataFrame

In [52]:
host = '10.11.40.228'
dbase = 'syncdb'
user = 'wfp'
pword = '75kw8uV9f9'
connection = psycopg2.connect(host=host,
                             database=dbase,
                             user=user,
                             password=pword)

cursor = connection.cursor()

cursor.execute("""SELECT table_name FROM information_schema.tables WHERE table_schema = 'wfp'""")
for table in cursor.fetchall():
    print(table)

###CLOSE THE CONNECTION!!!    
#connection.close()

('wfp_ph_airports_master_uuid',)
('wfp_ph_airports_slave_uuid',)
('audit_foreign',)
('claudio',)
('wld_poi_locations_ge',)
('location_resolution_submitted',)


In [53]:
### READ DATA
SQL = "SELECT * FROM wfp.wfp_ph_airports_slave_uuid"
server_data = gpd.read_postgis(SQL, connection, geom_col='shape', crs={'init':u'epsg:4326'},coerce_float=False)
#server_data.head(3)

In [54]:
geometry = server_data['shape']
crs = {'init': 'epsg:4326'}

gdf = GeoDataFrame(server_data,crs=crs,geometry=geometry)

#gdf.plot()

In [55]:
connection.close()

In [ ]:
#!pip install gsconfig
#https://github.com/boundlessgeo/gsconfig#installing

In [56]:
from sentinelsat.sentinel import SentinelAPI, read_geojson, geojson_to_wkt
from datetime import date

In [79]:
api = SentinelAPI('mgmanalili.updge', 'Suite#41', 'https://scihub.copernicus.eu/dhus')
print api

<sentinelsat.sentinel.SentinelAPI instance at 0x7fb700cd93b0>


In [117]:
data = '/home/michael/Desktop/map.geojson'
footprint = geojson_to_wkt(read_geojson(data))

In [119]:
products = api.query(footprint,
                    date = ('20161201', date(2016,12,28)),
                    platformname = 'Sentinel-1',
                    producttype='GRD')
products

OrderedDict([(u'6c5b562a-e7a9-4850-97d5-14e9ad887e7d',
              {u'acquisitiontype': u'NOMINAL',
               u'beginposition': datetime.datetime(2016, 12, 19, 21, 45, 43, 502000),
               u'endposition': datetime.datetime(2016, 12, 19, 21, 46, 8, 500000),
               u'filename': u'S1A_IW_GRDH_1SDV_20161219T214543_20161219T214608_014454_01772A_5EF6.SAFE',
               u'footprint': u'POLYGON ((122.595116 16.553761,120.278999 16.984495,120.575050 18.491655,122.911087 18.064365,122.595116 16.553761))',
               u'format': u'SAFE',
               u'gmlfootprint': u'<gml:Polygon srsName="http://www.opengis.net/gml/srs/epsg.xml#4326" xmlns:gml="http://www.opengis.net/gml">\n   <gml:outerBoundaryIs>\n      <gml:LinearRing>\n         <gml:coordinates>16.553761,122.595116 16.984495,120.278999 18.491655,120.575050 18.064365,122.911087 16.553761,122.595116</gml:coordinates>\n      </gml:LinearRing>\n   </gml:outerBoundaryIs>\n</gml:Polygon>',
               u'identifier

In [120]:
#api.to_geojson(products)
api.to_geodataframe(products)

,beginposition,orbitnumber,producttype,productclass,polarisationmode,platformname,size,acquisitiontype,instrumentshortname,title,...,slicenumber,link,uuid,instrumentname,missiondatatakeid,endposition,summary,link_icon,identifier,geometry
62c9fbff-76ca-4ce8-bd4f-bb5b06431dcf,2016-12-09 10:06:48.652,3318,GRD,S,VV,Sentinel-1,885.34 MB,NOMINAL,SAR-C SAR,S1B_IW_GRDH_1SSV_20161209T100648_20161209T1007...,...,8,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,62c9fbff-76ca-4ce8-bd4f-bb5b06431dcf,Synthetic Aperture Radar (C-band),23190,2016-12-09 10:07:15.982,"Date: 2016-12-09T10:06:48.652Z, Instrument: SA...",https://scihub.copernicus.eu/dhus/odata/v1/Pro...,S1B_IW_GRDH_1SSV_20161209T100648_20161209T1007...,"POLYGON ((118.924835 18.93848, 121.275772 19.3..."
6c5b562a-e7a9-4850-97d5-14e9ad887e7d,2016-12-19 21:45:43.502,14454,GRD,S,VV VH,Sentinel-1,1.57 GB,NOMINAL,SAR-C SAR,S1A_IW_GRDH_1SDV_20161219T214543_20161219T2146...,...,2,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,6c5b562a-e7a9-4850-97d5-14e9ad887e7d,Synthetic Aperture Radar (C-band),96042,2016-12-19 21:46:08.500,"Date: 2016-12-19T21:45:43.502Z, Instrument: SA...",https://scihub.copernicus.eu/dhus/odata/v1/Pro...,S1A_IW_GRDH_1SDV_20161219T214543_20161219T2146...,"POLYGON ((122.595116 16.553761, 120.278999 16...."
d86ea181-fc54-4793-8f01-6ce62b724d57,2016-12-09 10:06:23.653,3318,GRD,S,VV,Sentinel-1,809.7 MB,NOMINAL,SAR-C SAR,S1B_IW_GRDH_1SSV_20161209T100623_20161209T1006...,...,7,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,d86ea181-fc54-4793-8f01-6ce62b724d57,Synthetic Aperture Radar (C-band),23190,2016-12-09 10:06:48.651,"Date: 2016-12-09T10:06:23.653Z, Instrument: SA...",https://scihub.copernicus.eu/dhus/odata/v1/Pro...,S1B_IW_GRDH_1SSV_20161209T100623_20161209T1006...,"POLYGON ((119.268356 17.286638, 121.596085 17...."
f14ae480-7850-4cf1-bf0d-9e4afd00f17f,2016-12-07 21:45:43.955,14279,GRD,S,VV VH,Sentinel-1,1.57 GB,NOMINAL,SAR-C SAR,S1A_IW_GRDH_1SDV_20161207T214543_20161207T2146...,...,2,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,f14ae480-7850-4cf1-bf0d-9e4afd00f17f,Synthetic Aperture Radar (C-band),94632,2016-12-07 21:46:08.953,"Date: 2016-12-07T21:45:43.955Z, Instrument: SA...",https://scihub.copernicus.eu/dhus/odata/v1/Pro...,S1A_IW_GRDH_1SDV_20161207T214543_20161207T2146...,"POLYGON ((122.595306 16.553751, 120.279182 16...."
